In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import pickle

from moviepy.editor import VideoFileClip
from IPython.display import HTML

%matplotlib inline

In [ ]:
#Constant values imported from camera calibration and helper codes:

pickle_in1 = open("test_images/info_data.p","rb")

example_dict1 = pickle.load(pickle_in1)


mtx = example_dict1['mtx']
dist = example_dict1['dist']
M = example_dict1['M']
Minv = example_dict1['Minv']
lane_line_width_pixels = example_dict1['lane_line_width_pixels']
imshape = example_dict1['imshape']

#creating others Constant values to support function binary images.

orient = 'x'
Sobel_thresh = (30,120)
kernel_size = 5

rgb_channel = 'r'
rgb_thresh = (220,255)

hls_channel = 's'
hls_thresh = (180,255)

In [ ]:
def Undistorted_Images(img):
    return cv2.undistort(img, mtx, dist, None, mtx)

def abs_sobel_thresh(img,orient):
    
    # Apply the following steps to img
    #1) Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    #Applies a Gaussian Noise kernel
    blur =  cv2.GaussianBlur(gray, (kernel_size, kernel_size), 0)
    
    # 2) Take the derivative in x or y given orient = 'x' or 'y'
    if orient == 'x':
        sobel = cv2.Sobel(blur, cv2.CV_64F, 1, 0)
    if orient == 'y':
        sobel = cv2.Sobel(blur, cv2.CV_64F, 0, 1)

        
    # 3) Take the absolute value of the derivative or gradient
    abs_sobel = np.absolute(sobel)
    
    # 4) Scale to 8-bit (0 - 255) then convert to type = np.uint8
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))

    # 5) Create a mask of 1's where the scaled gradient magnitude 
            # is > thresh_min and < thresh_max
    sbinary = np.zeros_like(scaled_sobel)
    sbinary[(scaled_sobel >= Sobel_thresh[0]) & (scaled_sobel <= Sobel_thresh[1])] = 1

    # 6) Return this mask as your binary_output image

    return sbinary

def rgb_channel_binary(img,rgb_channel):
    # 1) Convert to HLS color space
    
    #hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    if rgb_channel == 'r':
        rgb_channel = img[:,:,0]
    if rgb_channel == 'g':
        rgb_channel = img[:,:,1]  
    if rgb_channel == 'b':
        rgb_channel = img[:,:,2]  
    # 2) Apply a threshold to the S channel
    
    binary = np.zeros_like(rgb_channel)
    binary[(rgb_channel > rgb_thresh[0]) & (rgb_channel <= rgb_thresh[1])] = 1

    
    # 3) Return a binary image of threshold result
    
    #binary_output = np.copy(img) # placeholder line
    rgb_binary =  binary
    
    return rgb_binary

def hls_channel_binary(img,hls_channel):
    # 1) Convert to HLS color space
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    
    if hls_channel == 'h':
        hls_channel = hls[:,:,0]
    if hls_channel == 'l':
        hls_channel = hls[:,:,1]  
    if hls_channel == 's':
        hls_channel = hls[:,:,2]  
    # 2) Apply a threshold to the S channel
    
    binary = np.zeros_like(hls_channel)
    binary[(hls_channel >= hls_thresh[0]) & (hls_channel <= hls_thresh[1])] = 1
    
    # 3) Return a binary image of threshold result

    hls_binary =  binary
    
    return hls_binary

def color_and_gradient(img):
    
    gradx = abs_sobel_thresh(img,orient='x')
    r_binary = rgb_channel_binary(img,'r')
    s_binary = hls_channel_binary(img,'s')
    
    # Stack each channel
    #color_binary = np.dstack(( np.zeros_like(gradx ),gradx ,s_binary  )) * 255
    
    combined_binary = np.zeros_like(gradx)
    combined_binary[(r_binary == 1)|(s_binary == 1) & (gradx == 1)] = 1
  
    return combined_binary 

def warp_images (img):
   
    return cv2.warpPerspective(img, M ,(imshape[1],imshape[0]))

In [ ]:
def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 20
    # Set the width of the windows +/- margin
    margin = 150
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        ### TO-DO: Find the four below boundaries of the window ###
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        
        ### TO-DO: Identify the nonzero pixels in x and y within the window ###
        good_left_inds =  ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        ### TO-DO: If you found > minpix pixels, recenter next window ###
        ### (`right` or `leftx_current`) on their mean position ###
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))        

        #pass # Remove this when you add your function

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img


def fit_polynomial(binary_warped):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    ### TO-DO: Fit a second order polynomial to each using `np.polyfit` ###
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]

    # Plots the left and right polynomials on the lane lines
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')

    #return out_img , left_fitx , right_fitx
    
    result = out_img

    return result , left_fit , right_fit , left_fitx , right_fitx,ploty

In [ ]:
def fit_poly(img_shape, leftx, lefty, rightx, righty):
    ### TO-DO: Fit a second order polynomial to each with np.polyfit() ###
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    ### TO-DO: Calc both polynomials using ploty, left_fit and right_fit ###
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    return left_fitx, right_fitx, ploty

def search_around_poly(binary_warped,left_fit,right_fit):

    # HYPERPARAMETER
    # Choose the width of the margin around the previous polynomial to search
    # The quiz grader expects 100 here, but feel free to tune on your own!
    margin = 100

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    ### TO-DO: Set the area of search based on activated x-values ###
    ### within the +/- margin of our polynomial function ###
    ### Hint: consider the window areas for the similarly named variables ###
    ### in the previous quiz, but change the windows to our new search area ###
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit new polynomials
    left_fitx, right_fitx, ploty = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)
    
    ## Visualization ##
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                              ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                              ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    
    # Plot the polynomial lines onto the image
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')
    ## End visualization steps ##
    
    return result , left_fit , right_fit , left_fitx , right_fitx, ploty    

In [ ]:
def draw_lines(binary_warped, left_fitx , right_fitx, ploty,undist):
    
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    
    
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (imshape[1], imshape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)

    return result

In [ ]:
def measure_curvature_real(ploty ,left_fit ,right_fit,left_fitx,right_fitx):
    '''
    Calculates the curvature of polynomial functions in meters.
    '''
    # Define conversions in x and y from pixels space to meters
    xm_per_pix = float(3.7/lane_line_width_pixels) # meters per pixel in x dimension 
    ym_per_pix = float(30./(720.)) # meters per pixel in y dimension
     
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    left_fitx = left_fitx[::-1]  # Reverse to match top-to-bottom in y
    right_fitx = right_fitx[::-1]  # Reverse to match top-to-bottom in y
    
    
    left_fit_cr = np.polyfit(ploty*ym_per_pix, left_fitx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, right_fitx*xm_per_pix, 2)
    
    ##### TO-DO: Implement the calculation of R_curve (radius of curvature) #####
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
       
    if left_curverad > 1500 or right_curverad > 1500:
        vehicle_curverad = 0
        direction = "straight line"        
    
    else:
        
        if left_curverad > right_curverad :
            vehicle_curverad = ((left_curverad - right_curverad)/2 )+ right_curverad
            direction = "to right"
        
        elif left_curverad < right_curverad :
            vehicle_curverad = ((right_curverad - left_curverad )/2) + left_curverad 
            direction = "to left"  
              
        else:
            vehicle_curverad = 0
            direction = "straight line"        
    
    return  vehicle_curverad , direction


def measure_offset(left_fitx , right_fitx):
    
    xm_per_pix = float(3.7/lane_line_width_pixels) # meters per pixel in x dimension      
        
    min_x = left_fitx[700]
    max_x = right_fitx[700]
    lane_line_center = ((max_x - min_x)/2)+ min_x
    
    if lane_line_center > (imshape[1]/2):
        offset = (lane_line_center - (imshape[1]/2))* xm_per_pix
        position = "to left"
    elif lane_line_center < imshape[1]/2:
        offset = (imshape[1]/2 - lane_line_center)* xm_per_pix
        position = "to Right" 
    else:
        offset = 0
        position = "In the middle"         
    
    return abs(offset) , position  

def add_info(img,vehicle_curverad , direction, offset , position):
    
    font = cv2.FONT_HERSHEY_COMPLEX_SMALL
    text1 = 'vehicle turn radius: ' + '{:04.1f}'.format(vehicle_curverad) + 'm' + ' {}'.format(direction)
    cv2.putText(img, text1, (40,50), font, 1.5, (255,255,255), 2, cv2.LINE_AA)
    text2 = 'Offset vehicle on the lane: ' + '{:.2f}'.format(abs(offset)) + 'm' + ' {}'.format(position)
    cv2.putText(img, text2, (40,100), font, 1.5, (255,255,255), 2, cv2.LINE_AA)
    
    return img  



## Creating functions   and Final Pipeline to draw lines and plot values considering video files.

In [ ]:
def process_image(img):
            
    undist = Undistorted_Images(img)
    img_warped = warp_images (undist)
    binary_warped = color_and_gradient(img_warped)

    pickle_in = open("output_images/previous_info.p","rb")
    input_data = pickle.load(pickle_in)
    in_list = input_data["in_list"] 
    
    left_fit_list = input_data["left_fit"]
    right_fit_list = input_data["right_fit"]    
    left_fitx_list = input_data["left_fitx"]
    right_fitx_list = input_data["right_fitx"]
    ploty_list = input_data["ploty"]

    left_fit = left_fit_list [-1]
    right_fit = right_fit_list [-1]
    try:
        left_fitx = left_fit_list[-1]
        right_fitx = right_fitx_list[-1]
    except:
        left_fitx = []
        right_fitx = []      
    ploty = ploty_list[-1]       
          
    n_frames = 3
    
    if len(in_list) < n_frames:
    
        result , nleft_fit , nright_fit , nleft_fitx , nright_fitx,nploty = fit_polynomial(binary_warped)
            
    else:
         
        result , nleft_fit , nright_fit , nleft_fitx , nright_fitx , nploty= search_around_poly(binary_warped,left_fit,right_fit)
        
    in_list.append(nleft_fit)
    left_fit_list.append(nleft_fit)
    right_fit_list.append(nright_fit)
    left_fitx_list.append(nleft_fitx)
    right_fitx_list.append(nright_fitx)
    ploty_list.append(nploty)
  

    #avg_left_fit = np.resize(np.median(left_fit_list[-n_frames:], axis = 0),3)
    #avg_right_fit = np.resize (np.median(right_fit_list[-n_frames:], axis = 0),3)    
    #avg_left_fitx = np.resize(np.median(left_fit_list[-n_frames:], axis = 0),720)
    #avg_right_fitx = np.resize(np.median(right_fit_list[-n_frames:], axis = 0),720 ) 
    #avg_ploty = np.resize(np.mean(ploty_list[-n_frames:], axis = 0),720 )     
   
    offset , position = measure_offset(nleft_fitx , nright_fitx)
    vehicle_curverad , direction = measure_curvature_real(nploty,nleft_fit,nright_fit,nleft_fitx,nright_fitx)
    
    real_image = draw_lines(binary_warped, nleft_fitx , nright_fitx, nploty,undist)
    final_image = add_info(real_image, vehicle_curverad , direction, offset , position )
   

    exp_pickle1 = {}
    exp_pickle1["left_fit"] = left_fit_list
    exp_pickle1["right_fit"] = right_fit_list
    exp_pickle1["left_fitx"] = left_fitx_list
    exp_pickle1["right_fitx"] = right_fitx_list
    exp_pickle1["in_list"] = in_list
    exp_pickle1["ploty"] = ploty_list
    pickle.dump( exp_pickle1, open( "output_images/previous_info.p", "wb" ) )  

    return final_image

In [ ]:
exp_pickle1 = {}
exp_pickle1["left_fit"] = [np.array([1,1,1])]
exp_pickle1["right_fit"] = [np.array([1,1,1])]
exp_pickle1["in_list"] = []
exp_pickle1["left_fitx"] = []
exp_pickle1["right_fitx"] = []
exp_pickle1["ploty"] = [np.array([1,1,1])]

pickle.dump( exp_pickle1, open( "output_images/previous_info.p", "wb" ) )

In [ ]:
white_output = 'output_images/project_video_Edu.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
#clip1 = VideoFileClip("project_video.mp4").subclip(0,5)
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))